In [0]:
# MAGIC %md
# MAGIC # 🏆 04_Gold_Aggregations
# MAGIC Construye tablas de hechos, dimensiones y agregados optimizados para BI y ML.
# MAGIC - Lee desde `fraude_qr.silver.qr_transactions`
# MAGIC - Crea tabla de hechos diaria: `gold.fact_qr_tx_daily`
# MAGIC - Crea dimensiones enriquecidas: `gold.dim_merchant`, `gold.dim_payer`
# MAGIC - Crea vista para BI: `gold.dash_daily_kpis`
# MAGIC - Particiona y optimiza para rendimiento.

# COMMAND ----------

from pyspark.sql.functions import col, count, sum, avg, max, date_trunc, current_date, when, lit, stddev

# --- 1. Configuración ---
silver_table = "fraude_qr.silver.qr_transactions"
gold_fact_table = "fraude_qr.gold.fact_qr_tx_daily"
gold_dim_merchant_table = "fraude_qr.gold.dim_merchant"
gold_dim_payer_table = "fraude_qr.gold.dim_payer"
gold_dash_table = "fraude_qr.gold.dash_daily_kpis"

# --- 2. Leer desde Silver ---
print(f"📂 Leyendo datos desde: {silver_table}")
df_silver = spark.table(silver_table)
print(f"📊 Registros en Silver: {df_silver.count():,}")

# --- 3. Crear Tabla de Hechos: Transacciones Diarias ---
print("\n🧱 Construyendo tabla de hechos: fact_qr_tx_daily...")

df_fact_daily = (
    df_silver
    .withColumn("date_key", date_trunc("day", col("created_at")).cast("date"))
    .groupBy("date_key", "merchant_id", "currency", "channel", "qr_type")
    .agg(
        count("*").alias("tx_count"),
        sum("amount").alias("amount_sum"),
        sum(when(col("is_fraud") == 1, 1).otherwise(0)).alias("fraud_count"),
        sum(when(col("is_fraud") == 1, col("amount")).otherwise(0)).alias("fraud_amount_sum"),
        avg("amount").alias("avg_amount"),
        stddev("amount").alias("stddev_amount"),  # Útil para detección de anomalías
        max("created_at").alias("last_tx_time")
    )
    .withColumn("fraud_rate", 
        when(col("tx_count") > 0, col("fraud_count") / col("tx_count")).otherwise(0)
    )
    .withColumn("fraud_ratio_amount", 
        when(col("amount_sum") > 0, col("fraud_amount_sum") / col("amount_sum")).otherwise(0)
    )
)

# Escribir tabla de hechos (particionada por date_key para rendimiento)
(
    df_fact_daily
    .write
    .mode("overwrite")
    .partitionBy("date_key")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_fact_table)
)

print(f"✅ Tabla de hechos guardada: {gold_fact_table}")

# --- 4. Crear Dimensión: Merchants ---
print("\n🏷️ Construyendo dimensión: dim_merchant...")

df_dim_merchant = (
    df_silver
    .groupBy("merchant_id")
    .agg(
        count("*").alias("total_tx_last_30d"),
        sum("amount").alias("total_amount_last_30d"),
        avg("amount").alias("avg_amount"),
        stddev("amount").alias("stddev_amount"),
        sum(col("is_fraud")).alias("fraud_count_last_30d"),
        max("created_at").alias("last_tx_date"),
        max("merchant_lat").alias("merchant_lat"),  # Asumimos que no cambia
        max("merchant_lon").alias("merchant_lon")
    )
    .withColumn("fraud_rate", 
        when(col("total_tx_last_30d") > 0, 
             col("fraud_count_last_30d") / col("total_tx_last_30d")
        ).otherwise(0)
    )
    .withColumn("risk_tier", 
        when(col("fraud_rate") > 0.05, "high")
        .when(col("fraud_rate") > 0.01, "medium")
        .otherwise("low")
    )
)

# Escribir dimensión
(
    df_dim_merchant
    .write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_dim_merchant_table)
)

print(f"✅ Dimensión guardada: {gold_dim_merchant_table}")

# --- 5. Crear Dimensión: Payers ---
print("\n🏷️ Construyendo dimensión: dim_payer...")

df_dim_payer = (
    df_silver
    .groupBy("payer_id")
    .agg(
        count("*").alias("total_tx_last_30d"),
        sum("amount").alias("total_amount_last_30d"),
        avg("amount").alias("avg_amount"),
        sum(col("is_fraud")).alias("fraud_count_last_30d"),
        max("created_at").alias("last_tx_date")
    )
    .withColumn("fraud_rate", 
        when(col("total_tx_last_30d") > 0, 
             col("fraud_count_last_30d") / col("total_tx_last_30d")
        ).otherwise(0)
    )
    .withColumn("segment", 
        when(col("total_amount_last_30d") > 100000, "vip")
        .when(col("total_amount_last_30d") > 10000, "regular")
        .otherwise("new")
    )
)

(
    df_dim_payer
    .write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_dim_payer_table)
)

print(f"✅ Dimensión guardada: {gold_dim_payer_table}")

# --- 6. Crear Vista para BI: dash_daily_kpis ---
print("\n📈 Creando vista para BI: dash_daily_kpis...")

df_dash_kpis = (
    spark.table(gold_fact_table)
    .select(
        "date_key",
        "tx_count",
        "fraud_count",
        "amount_sum",
        "fraud_amount_sum",
        "fraud_rate",
        "fraud_ratio_amount",
        "channel",
        "qr_type"
    )
    .withColumn("expected_loss", col("fraud_ratio_amount") * col("amount_sum"))
)

(
    df_dash_kpis
    .write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(gold_dash_table)
)

print(f"✅ Vista para BI guardada: {gold_dash_table}")

# --- 7. Verificación Final ---
print("\n🔍 Muestra de fact_qr_tx_daily:")
spark.table(gold_fact_table).limit(5).display()

print(f"\n📊 Total días en fact_qr_tx_daily: {spark.table(gold_fact_table).select('date_key').distinct().count()}")

print("\n🔍 Muestra de dim_merchant:")
spark.table(gold_dim_merchant_table).limit(5).display()

print("\n🔍 Muestra de dash_daily_kpis:")
spark.table(gold_dash_table).limit(5).display()

📂 Leyendo datos desde: fraude_qr.silver.qr_transactions
📊 Registros en Silver: 7,362,604

🧱 Construyendo tabla de hechos: fact_qr_tx_daily...
✅ Tabla de hechos guardada: fraude_qr.gold.fact_qr_tx_daily

🏷️ Construyendo dimensión: dim_merchant...
✅ Dimensión guardada: fraude_qr.gold.dim_merchant

🏷️ Construyendo dimensión: dim_payer...
✅ Dimensión guardada: fraude_qr.gold.dim_payer

📈 Creando vista para BI: dash_daily_kpis...
✅ Vista para BI guardada: fraude_qr.gold.dash_daily_kpis

🔍 Muestra de fact_qr_tx_daily:


date_key,merchant_id,currency,channel,qr_type,tx_count,amount_sum,fraud_count,fraud_amount_sum,avg_amount,stddev_amount,last_tx_time,fraud_rate,fraud_ratio_amount
1970-01-04,63,USD,app,dynamic,3,181452.03,0,0.0,60484.01,102903.12540219029,1970-01-04T23:00:00.000Z,0.0,0.0
1970-01-04,5227,USD,app,dynamic,6,223179.89,0,0.0,37196.64833333334,60677.68917921955,1970-01-04T23:00:00.000Z,0.0,0.0
1970-01-04,383,USD,app,dynamic,1,19696.0,0,0.0,19696.0,null,1970-01-04T21:00:00.000Z,0.0,0.0
1970-01-04,4406,USD,app,dynamic,7,373380.36999999994,0,0.0,53340.05285714285,59330.27155182159,1970-01-04T23:00:00.000Z,0.0,0.0
1970-01-04,4029,USD,app,dynamic,3,789023.04,0,0.0,263007.68,404625.2904435988,1970-01-04T22:00:00.000Z,0.0,0.0



📊 Total días en fact_qr_tx_daily: 61

🔍 Muestra de dim_merchant:


merchant_id,total_tx_last_30d,total_amount_last_30d,avg_amount,stddev_amount,fraud_count_last_30d,last_tx_date,merchant_lat,merchant_lon,fraud_rate,risk_tier
271,619,9.316098870310569E7,150502.4050130948,545898.1670205811,0,2025-08-30T21:21:09.000Z,-34.23,-58.129999999999995,0.0,low
1785,815,1.2127584721115255E8,148804.72050448167,357551.0743866862,1,2025-08-30T20:13:55.000Z,-33.716,-57.616,0.001226993865030675,low
9762,705,1.5735216975036043E8,223194.56702178784,1067173.2772922984,1,2025-08-30T19:29:35.000Z,-33.739,-57.638999999999996,0.0014184397163120568,low
2851,727,1.0062495330318904E8,138411.214997509,343855.4277300972,0,2025-08-30T10:52:57.000Z,-33.65,-57.55,0.0,low
1898,711,1.2763948803800723E8,179521.08022223238,776429.588846741,1,2025-08-30T22:04:47.000Z,-33.603,-57.503,0.0014064697609001407,low



🔍 Muestra de dash_daily_kpis:


date_key,tx_count,fraud_count,amount_sum,fraud_amount_sum,fraud_rate,fraud_ratio_amount,channel,qr_type,expected_loss
1970-01-01,26,0,1944858.3400000003,0.0,0.0,0.0,app,dynamic,0.0
1970-01-01,71,0,1.119811859E7,0.0,0.0,0.0,app,dynamic,0.0
1970-01-01,27,1,983647.0300000001,170.0,0.037037037037037035,1.728262220239713E-4,app,dynamic,170.0
1970-01-01,41,0,1.0504994600000001E7,0.0,0.0,0.0,app,dynamic,0.0
1970-01-01,51,0,7582524.5699999975,0.0,0.0,0.0,app,dynamic,0.0
